In [ ]:
import streamlit as st
import pandas as pd
import joblib
import numpy as np

# Charger le modèle et les encodeurs
model = joblib.load('xgboost_churn_model.pkl')
scaler = joblib.load('scaler.pkl')
label_encoders = joblib.load('label_encoders.pkl')

# Titre de l'application
st.title("Prédiction de Désabonnement (Churn) - Expresso Telecom")
st.markdown("""
Cette application prédit si un client risque de se désabonner (Churn = 1) ou non (Churn = 0).
""")

# Créer un formulaire pour les features
st.sidebar.header("Paramètres du Client")

# Fonction pour inverser l'encodage (affichage des labels)
def decode_label(encoder, value):
    return encoder.classes_[value]

# Collecter les inputs utilisateur
region = st.sidebar.selectbox("Région", label_encoders['REGION'].classes_)
tenure = st.sidebar.selectbox("Ancienneté (TENURE)", label_encoders['TENURE'].classes_)
montant = st.sidebar.number_input("Montant dépensé (MONTANT)", min_value=0.0, value=5000.0)
frequence_rech = st.sidebar.number_input("Fréquence de recharge (FREQUENCE_RECH)", min_value=0.0, value=5.0)
revenue = st.sidebar.number_input("Revenu (REVENUE)", min_value=0.0, value=5000.0)
arpu_segment = st.sidebar.number_input("ARPU Segment", min_value=0.0, value=1000.0)
frequence = st.sidebar.number_input("Fréquence d'utilisation (FREQUENCE)", min_value=0.0, value=10.0)
data_volume = st.sidebar.number_input("Volume de données (DATA_VOLUME)", min_value=0.0, value=1000.0)
on_net = st.sidebar.number_input("Appels On-Net (ON_NET)", min_value=0.0, value=100.0)
orange = st.sidebar.number_input("Appels vers Orange (ORANGE)", min_value=0.0, value=50.0)
tigo = st.sidebar.number_input("Appels vers Tigo (TIGO)", min_value=0.0, value=10.0)
zone1 = st.sidebar.number_input("Zone 1 (ZONE1)", min_value=0.0, value=1.0)
zone2 = st.sidebar.number_input("Zone 2 (ZONE2)", min_value=0.0, value=1.0)
mrg = st.sidebar.selectbox("MRG", label_encoders['MRG'].classes_)
regularity = st.sidebar.number_input("Régularité (REGULARITY)", min_value=0.0, value=30.0)
top_pack = st.sidebar.selectbox("Forfait Principal (TOP_PACK)", label_encoders['TOP_PACK'].classes_)
freq_top_pack = st.sidebar.number_input("Fréquence d'utilisation du forfait (FREQ_TOP_PACK)", min_value=0.0, value=5.0)

# Bouton de prédiction
if st.sidebar.button("Prédire le Churn"):
    # Encoder les variables catégorielles
    region_encoded = label_encoders['REGION'].transform([region])[0]
    tenure_encoded = label_encoders['TENURE'].transform([tenure])[0]
    mrg_encoded = label_encoders['MRG'].transform([mrg])[0]
    top_pack_encoded = label_encoders['TOP_PACK'].transform([top_pack])[0]

    # Créer un DataFrame avec les inputs
    input_data = pd.DataFrame({
        'REGION': [region_encoded],
        'TENURE': [tenure_encoded],
        'MONTANT': [montant],
        'FREQUENCE_RECH': [frequence_rech],
        'REVENUE': [revenue],
        'ARPU_SEGMENT': [arpu_segment],
        'FREQUENCE': [frequence],
        'DATA_VOLUME': [data_volume],
        'ON_NET': [on_net],
        'ORANGE': [orange],
        'TIGO': [tigo],
        'ZONE1': [zone1],
        'ZONE2': [zone2],
        'MRG': [mrg_encoded],
        'REGULARITY': [regularity],
        'TOP_PACK': [top_pack_encoded],
        'FREQ_TOP_PACK': [freq_top_pack]
    })

    # Scaler les données
    input_scaled = scaler.transform(input_data)

    # Prédiction
    prediction = model.predict(input_scaled)[0]
    probability = model.predict_proba(input_scaled)[0][1]

    # Affichage des résultats
    st.subheader("Résultat de la Prédiction")
    if prediction == 1:
        st.error(f"⚠️ **Risque élevé de désabonnement (Churn = 1)**")
        st.write(f"Probabilité de désabonnement: **{probability:.2%}**")
    else:
        st.success(f"✅ **Faible risque de désabonnement (Churn = 0)**")
        st.write(f"Probabilité de désabonnement: **{probability:.2%}**")

    # Afficher les détails du client
    st.subheader("Détails du Client")
    st.write(input_data)